In [7]:
import pandahouse as ph

In [8]:
connection_default = {'host': 'http://clickhouse.beslan.pro:8080',
                      'database': 'default',
                      'user': 'student', 
                      'password': 'dpo_python_2020'}

In [9]:
query = """
SELECT 
    time,
    COUNT() success_studs
FROM(
    SELECT 
        toStartOfMonth(toDateTime(timest)) as time,
        st_id,
        COUNT() as count_ex
    FROM default.peas
    WHERE correct == true
    GROUP BY time, st_id
    ORDER BY time DESC
    ) as tab
WHERE tab.count_ex >=20
GROUP BY time
"""
success_studs = ph.read_clickhouse(query, connection=connection_default)
success_studs

,time,success_studs
0,2021-10-01,136


In [14]:
query = """
WITH 
    payed_studs as
    (SELECT 
        uniqExact(st_id) as payed_studs_count,
        SUM(money) as money_sum,
        test_grp 
    FROM default.final_project_check 
    JOIN default.studs 
    ON default.final_project_check.st_id == default.studs.st_id 
    GROUP BY test_grp
    ),
    
    all_studs as 
    (SELECT 
        uniqExact(st_id) as all_studs_count, 
        test_grp 
    FROM default.studs
    GROUP BY test_grp
    ),
    
    active_studs_table as 
    (SELECT 
        st_id,
        count(*) > 10 as active,
        countIf(st_id, subject == 'Math') >= 2 as active_math,
        min(default.studs.test_grp) as test_grp
    FROM default.peas
    LEFT JOIN default.studs 
    ON default.peas.st_id == default.studs.st_id
    WHERE correct == 1 
    GROUP BY st_id
    ),
    
    active_studs as 
    (SELECT 
        test_grp,
        SUM(active) as active_studs_count,
        SUM(active_math) as active_math_studs_count
    FROM active_studs_table
    GROUP BY test_grp
    ),
    
    payed_active_studs as
    (SELECT
        test_grp,
        SUM(CASE WHEN active == 1 THEN money ELSE 0 end) as active_sum_money,
        countIf(DISTINCT(active_studs_table.st_id), active == 1) as payed_active_studs_count,
        countIf(DISTINCT(active_studs_table.st_id), active_math == 1) as payed_active_math_studs_count
    FROM active_studs_table
    INNER JOIN default.final_project_check
    ON active_studs_table.st_id == default.final_project_check.st_id
    GROUP BY test_grp
    )

SELECT 
    payed_studs.test_grp as group,
    payed_studs_count / all_studs_count as cr,
    payed_active_studs_count / active_studs_count as cr_active,
    payed_active_math_studs_count / active_math_studs_count as cr_active_math,
    cr * money_sum / payed_studs_count  as arpu,
    cr_active * active_sum_money / payed_active_studs_count  as arpau
    
FROM payed_studs

JOIN all_studs 
ON payed_studs.test_grp == all_studs.test_grp
JOIN active_studs
ON payed_studs.test_grp == active_studs.test_grp
JOIN payed_active_studs
ON payed_studs.test_grp == payed_active_studs.test_grp
"""
studs_metrics = ph.read_clickhouse(query, connection=connection_default)
studs_metrics

,group,cr,cr_active,cr_active_math,arpu,arpau
0,control,0.049180,0.110236,0.102041,4540.983607,10393.700787
1,pilot,0.108475,0.260417,0.142857,11508.474576,29739.583333


По результатам А/B теста все основные метрики в тестовой группе имеют показатели выше аналогичных в контрольной группе.